# Mapa Interactivo de Promedios Climáticos por Año y predicción del año 2026
Este módulo permite visualizar, analizar y simular variables climáticas registradas por estaciones meteorológicas mediante:

🔹Mapas interactivos con Folium

🔹Gráficas resumen anuales

🔹Una simulación del año 2026 basada en datos históricos

🔹El usuario puede seleccionar un año histórico o generar una proyección simulada para 2026 usando promedios del último año disponible.







🎛️ Interfaz de Usuario

El sistema cuenta con los siguientes controles interactivos:

🔹 Selector de Año

El cual nos permite elegir un año histórico disponible en el dataset.

📈 Gráfica General Anual

La gráfica permite comparar rápidamente:

Temperatura

Humedad

Lluvia

Viento

para un mismo año, facilitando el análisis climático global.


-------------------Selección de Año y Variable------------------


A través de los widgets interactivos:

Selector de Año: permite analizar un año específico del dataset.

Selector de Variable: define qué variable climática se analizará en el mapa.

Cada cambio actualiza automáticamente los resultados mostrados.

----------------Promedio Anual por Estación-----------

Una vez seleccionado el año:

El sistema filtra los datos correspondientes.

Calcula el promedio anual de la variable seleccionada para cada estación meteorológica.

Solo se consideran estaciones con datos válidos y coordenadas geográficas.

Este promedio representa el comportamiento general de la variable durante todo el año en cada punto del país.

📌 Información mostrada al hacer clic en una estación

Cada marcador incluye un popup con:

🏷️ Nombre de la estación

📅 Año analizado

📊 Variable climática seleccionada

📈 Promedio anual calculado

--------------Visualización en Mapa Interactivo (Folium)-----------

Cada estación se representa mediante un marcador circular en el mapa.

🎨 Colores de los marcadores

Los colores se asignan automáticamente según la distribución de los valores:

| Color      | Significado    |
| ---------- | -------------- |
| 🟢 Verde   | Valores bajos  |
| 🟠 Naranja | Valores medios |
| 🔴 Rojo    | Valores altos  |


--------------- Inicio del codigo -------------------------

Imports para manejar los valores
pandas: manejar el dataset

folium: crear mapas interactivos

ipywidgets: crear formularios (selectores)

display / clear_output: actualizar el contenido dinámicamente

Path: manejar rutas del proyecto de forma segura

In [1]:
import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from pathlib import Path

Carga el dataset climático

Convierte timestamp a formato fecha

Usa rutas relativas

In [2]:
BASE_DIR = Path('..').resolve()
csv_path = BASE_DIR / 'data_clean' / 'master' / 'master_dataset_final_wide.csv'

df = pd.read_csv(csv_path, parse_dates=['timestamp'])

Extrae el año, que será usado para:

Filtrar datos

Comparar años

Visualizar evolución temporal

In [3]:
# Variables temporales
df["year"] = df["timestamp"].dt.year

Define qué puede elegir el usuario
Esto evita errores y mantiene el análisis controlado.

In [4]:
variables = ["TEMP", "HUMEDAD", "LLUVIA", "VIENTO"]


mediante esto se puede:

Seleccionar un año

Seleccionar una variable climática

Esto convierte el notebook en una herramienta interactiva.

In [ ]:
years = sorted(df["year"].dropna().unique().tolist())
if 2026 not in years:
    years.append(2026)

year_selector = widgets.Dropdown(
    options=years,
    description="Año:"
)

variable_selector = widgets.Dropdown(
    options=variables,
    description="Variable:"
)

output = widgets.Output()

In [6]:

def generar_mapa():
    with output:
        clear_output()

        year = year_selector.value
        var = variable_selector.value

        df_f = df[df["year"] == year]

        if df_f.empty:
            print("No hay datos para ese año")
            return

        station_avg = (
            df_f
            .dropna(subset=[var, "lat", "lon"])
            .groupby(["station_id", "station_name", "lat", "lon"])[var]
            .mean()
            .reset_index()
        )

        center = [station_avg["lat"].median(), station_avg["lon"].median()]
        m = folium.Map(location=center, zoom_start=7)

        q1, q2 = station_avg[var].quantile([0.33, 0.66])

        for _, r in station_avg.iterrows():
            value = r[var]
            color = "green" if value < q1 else "orange" if value < q2 else "red"

            folium.CircleMarker(
                [r["lat"], r["lon"]],
                radius=8,
                color=color,
                fill=True,
                fill_opacity=0.7,
                popup=f"""
                <b>{r['station_name']}</b><br>
                Año: {year}<br>
                {var}: {value:.2f}
                """
            ).add_to(m)

        display(m)

        # Gráfica general
        general_avg = df_f[variables].mean()
        plt.figure(figsize=(8,4))
        plt.bar(general_avg.index, general_avg.values)
        plt.title(f"Promedio anual ({year})")
        plt.grid(axis="y")
        plt.show()


def generar_mapa_2026():
    with output:
        clear_output()

        var = variable_selector.value
        df_2025 = df[df["year"] == 2025]

        if df_2025.empty:
            print("No hay datos para 2025")
            return

        station_avg = (
            df_2025
            .dropna(subset=[var, "lat", "lon"])
            .groupby(["station_id", "station_name", "lat", "lon"])[var]
            .mean()
            .reset_index()
        )

        np.random.seed(42)
        station_avg[var] *= (1 + np.random.normal(0, 0.03, len(station_avg)))

        center = [station_avg["lat"].median(), station_avg["lon"].median()]
        m = folium.Map(location=center, zoom_start=7)

        q1, q2 = station_avg[var].quantile([0.33, 0.66])

        for _, r in station_avg.iterrows():
            value = r[var]
            color = "green" if value < q1 else "orange" if value < q2 else "red"

            folium.CircleMarker(
                [r["lat"], r["lon"]],
                radius=8,
                color=color,
                fill=True,
                fill_opacity=0.7,
                popup=f"""
                <b>{r['station_name']}</b><br>
                Año: 2026 (simulado)<br>
                {var}: {value:.2f}
                """
            ).add_to(m)

        display(m)

        plt.figure(figsize=(6,4))
        plt.bar(["2026"], [station_avg[var].mean()])
        plt.title(f"Promedio estimado {var} - 2026")
        plt.grid(axis="y")
        plt.show()

# CONTROLADOR (DECIDE QUÉ MAPA MOSTRAR)

def manejar_cambio(change=None):
    if year_selector.value == 2026:
        generar_mapa_2026()
    else:
        generar_mapa()

year_selector.observe(manejar_cambio, names="value")
variable_selector.observe(manejar_cambio, names="value")

display(year_selector, variable_selector, output)

# CARGA INICIAL
manejar_cambio()

Dropdown(description='Año:', options=(2025, 2026), value=2025)

Dropdown(description='Variable:', options=('TEMP', 'HUMEDAD', 'LLUVIA', 'VIENTO'), value='TEMP')

Output()

INTERPRETACIÓN de la grafica 2026
--------------------------------
El año 2026 se estima mediante una simulación basada en el comportamiento promedio del último año disponible (2025), incorporando una variación aleatoria controlada.
Debido a que el modelo no introduce cambios estructurales (eventos extremos o tendencias climáticas abruptas), los valores simulados tienden a mantenerse cercanos a los niveles observados en 2025.